# Notes on the sampling of weighted graphs for MIS estimation

In [ ]:
%pylab inline

### No weights on the nodes (tasks)

Let us define $$G=(V,E,\omega)$$ as a weighted graph on the vertex set V, with the complete edgeset $$E = V \times V$$ weighted by the weight function $$\omega: E \to \mathbf{R}$$

Let us further consider the weight function as inducing a probability on the edges by normalizing the weight function in $(0,1)$, $$\tilde{\omega} = \frac{\omega}{\max_E{\omega}}$$ graphs in the following way. To a certain instanced graph 
$$g = (V,E_g) \in W$$
where $W$ is the space of all graphs on the nodeset $V$, we associate the probability $p(g)$:    
$$p(g) = \prod_{e \in E_g} \tilde{\omega_e} \prod_{e \in E \setminus E_g} (1-\tilde{\omega_e})$$  

On each graph $g$ one can then calculate the MIS and obtain the corresponding (normalized) maximum independence number $\theta : W \to \mathbf{R}$ and finally try to calculate $E_G[\theta]$ in the form:
$$ E_G[\theta] = \sum_g \theta(g)p(g) $$  
There are two problems:
1. how to sample the space of graphs in a way that is meaningful
2. substitute the analytical expression for $\theta$ in the sum. 

**Proposal for the sampling: ** $p(g)$ is written as a product of independent, uncorrelated probabilities at this stage (even if we consider the possibility to generalized $\omega$ to more refined functions). If we keep this assumption, then the generation of the instanced graphs becomes very straightforward:  

- for each possible edge, flip a coin;
- with probability $\omega_e$ add the edge, otherwise do nothing;

The total probability of the graph is that of all the edges being present. 
In this way it becomes possible to do MC simulations, which in the crudest form looks like this:
- generate a sample $g_i$, $i=1,2,\ldots,n$ from $p_g$
- calculate $I = \frac{1}{n} \sum_{i=0}^n \theta(g_i)$
- by LOLN $I$ is guaranteed to converge to $I \to \sum_g \theta(g) p(g)$

NOTE: one needs to keep the variance under control!! 

**Proposal for the independence number estimate: **
Basically, if we can prove that the resulting graph tends to be Poissonian,
we calculate the density of $g$ and plug it in the formula given by *Ricci-Tersenghi et al.*

The function below (also included in IGTools.py) can be used to produce binary graphs from a weighted graph.  
In the current form it uses the dumbest possible normalization from weights to probabilities (*normalization by the max weight*) and the most basic probability distribution, but it can readily be generalized to more complex cases. 

In [5]:
def generate_graph_instance(G):
    import networkx as nx;
    from random import random;
    g = nx.Graph();
    g.add_nodes_from(G.nodes());
    w = nx.get_edge_attributes(G,'weight');
    ## this is only the trivial normalization on the maximum edge weight
    ## which implies that edge will always be present.
    ## different normalizations can easily implemented
    w = dict(zip(w.keys(), np.array(w.values()) / float(np.max(w.values()))));
    for e in w:
        if random()<w[e]:
            g.add_edge(e[0],e[1]);
    return g;

### Weights on the nodes (tasks)

- add Biswadip's proposal about optimization